In [1]:
%run parseData.ipynb
#%run dataClass.ipynb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [81]:
class GestureModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 48, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(0.3)

        self.conv2 = nn.Conv1d(48, 48, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(24, 12)
        self.relu3 = nn.ReLU()
        self.do3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(12, 4)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.do1(x)

        x = self.relu2(self.conv2(x))
        x = self.pool2(x)

        x = self.flat(x)

        print(x.shape())

        x = self.relu3(self.fc3(x))
        x = self.do3(x)

        x = self.fc4(x)

        return x

In [2]:
path = "./data/"

In [108]:
def MultiplyVector(vector_data):
    multiplied_list = []
    for data in vector_data:
        multiplied_vectors = []
        for vector in data:
            multiplied = np.prod(vector)
            multiplied_vectors.append(multiplied)
        multiplied_list.append(multiplied_vectors)
    return multiplied_list

In [ ]:
all_data = read_data(path)
data = torch.from_numpy(all_data)

#data, data.shape

In [42]:
labels = data[:, :1].long()

labels.reshape((-1,))

#labels_encoding = torch.zeros(labels.shape[0], 4)
#labels_encoding = labels.unsqueeze(1)
#labels_encoding.scatter_(1, labels.unsqueeze(1), 1.0)

#labels_encoding, labels_encoding.shape

labels, labels.shape

(tensor([[0],
         [2],
         [2],
         [2],
         [0],
         [0],
         [0],
         [3],
         [2],
         [2],
         [3],
         [2],
         [1],
         [1],
         [0],
         [2],
         [0],
         [2],
         [0],
         [1],
         [3],
         [2],
         [3],
         [3],
         [2],
         [3],
         [0],
         [1],
         [1],
         [2],
         [0],
         [0],
         [0],
         [0],
         [1],
         [3],
         [0],
         [3],
         [2],
         [3],
         [2],
         [2],
         [3],
         [0],
         [3],
         [2],
         [2],
         [3],
         [1],
         [2],
         [1],
         [1],
         [2],
         [0],
         [2],
         [0],
         [1],
         [3],
         [3],
         [0],
         [3],
         [2],
         [1],
         [1],
         [3],
         [2],
         [0],
         [2],
         [2],
         [2],
         [2],
      

In [ ]:
rotations = data[:, 2:]

#rotations, rotations.shape

In [22]:
split = round(len(data) * 0.8)
split

113

In [ ]:
train_data = rotations[:split]
train_labels = labels[:split]

#train_data, train_data.shape, train_labels, train_labels.shape

In [ ]:
test_data = rotations[split:]
test_labels = labels[split:]

#test_data, test_data.shape, test_labels, test_labels.shape

In [105]:
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

In [104]:
model = GestureModel()
model.float()
optimizer = optim.SGD(model.parameters(), lr=0.001)

n_epochs = 20

for epoch in range(n_epochs):

    total_loss = 0

    for inputs, labels in train_loader:
        pred = model(inputs)
        loss = (pred - labels) ** 2

        optimizer.zero_grad()
        loss.backward()

        total_loss += loss
        
        optimizer.step()
    acc = 0
    count = 0
    for inputs, labels in test_loader:
        pred = model(inputs)
        acc += (torch.argmax(pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%") % (epoch, acc * 100)

tensor([[0.2812, 0.2812, 0.2552, 0.2995, 0.2713, 0.2500, 0.4432, 0.0298, 0.0167,
         0.4144, 0.0338, 0.0163, 0.2667, 0.0367, 0.0201, 0.2796, 0.1986, 0.0399,
         0.0236, 0.2500, 0.0167, 0.0163, 0.0201, 0.0236]], dtype=torch.float64)


RuntimeError: expected scalar type Double but found Float